<a href="https://colab.research.google.com/github/nonamesims4/The-Sims-4-MOD-JSON-/blob/main/sims4json%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E7%BF%BB%E8%A8%B3%E4%BF%AE%E6%AD%A3%E3%83%84%E3%83%BC%E3%83%AB%E5%85%B1%E6%9C%89%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab用: The Sims 4 MOD用JSON一括日本語化スクリプト
# 日本語のみの文章はスキップするがタグ置換＆s4 stbl mergeの翻訳語の先頭アスタリスク削除は必ず行う
# 英語と日本語混在のみ翻訳実施。プレースホルダー保護＆強化復元済み。
# 1セル完結・丁寧なコメント付き

# deep-translator インストール（初回のみ）
!pip install deep-translator

from google.colab import files
import json
import re
from deep_translator import GoogleTranslator
import time
from IPython.display import FileLink, display

# JSONファイルアップロード（完了で処理開始）
uploaded = files.upload()
input_path = next(iter(uploaded))

# プレースホルダーを安全なトークンに置換（タグ保護）
def mask_placeholders(text):
    # {}内に英数字・アンダースコア・ドット・アポストロフィ・ハイフンを許可
    pattern = r'\{[0-9a-zA-Z\._\'\-]+\}'
    found = re.findall(pattern, text)
    masked = text
    placeholder_map = {}
    for i, ph in enumerate(found):
        token = f'__PH_{i}__'   # スペースなし統一トークン
        masked = masked.replace(ph, token, 1)
        placeholder_map[token] = ph
    return masked, placeholder_map

# トークンの分割や空白混入でも復元可能なアンマスク関数
def unmask_placeholders(text, placeholder_map):
    for token, ph in placeholder_map.items():
        # 各文字の間に任意の空白が入る可能性に対応
        escaped_chars = list(re.escape(c) for c in token)
        pattern_str = r'\s*'.join(escaped_chars)
        pattern = re.compile(pattern_str, re.MULTILINE)
        text = pattern.sub(ph, text)
    return text

# タグの英語→日本語置換辞書
replace_dict = {
    '{M0.he}': '{M0.彼}',
    '{M0.his}': '{M0.彼の}',
    '{F0.she}': '{F0.彼女}',
    '{F0.her}': '{F0.彼女の}',
    '{M1.he}': '{M1.彼}',
    '{M1.his}': '{M1.彼の}',
    '{F1.she}': '{F1.彼女}',
    '{F1.her}': '{F1.彼女の}',

    '{m0.he}': '{M0.彼}',
    '{m0.his}': '{M0.彼の}',
    '{f0.she}': '{F0.彼女}',
    '{f0.her}': '{F0.彼女の}',
    '{m1.he}': '{M1.彼}',
    '{m1.his}': '{M1.彼の}',
    '{f1.she}': '{F1.彼女}',
    '{f1.her}': '{F1.彼女の}',

    "{M0.he's}": "{M0.彼の}",
    "{F0.she's}": "{F0.彼女の}",

    '{0.simfirstname}': '{0.SimFirstName}',
    '{1.simfirstname}': '{1.SimFirstName}',

    '{0.SimPronounSubjective}': '{M0.彼}{F0.彼女}',
    '{0.SimPronounPossessiveDependent}': '{M0.彼}{F0.彼女}',
    '{0.SimPronounReflexive}': '{M0.彼}{F0.彼女}',
    '{0.SimPronounObjective}': '{M0.彼}{F0.彼女}',

    '{F0.彼女の}{F0.彼女}': '{F0.彼女}',
    '{F0.彼女}{F0.彼女の}': '{F0.彼女}',
    '{M0.彼の}{M0.彼}': '{M0.彼}',
    '{M0.彼}{M0.彼の}': '{M0.彼}',

    '{F0.彼女の}の本': '{F0.彼女}の本',
    '{M0.彼の}の家': '{M0.彼}の家',

    '{F0.彼女に}に本': '{F0.彼女}に本',
    '{M0.彼に}に家': '{M0.彼}に家',
    '{F0.彼女が}が本': '{F0.彼女}が本',
    '{M0.彼が}が家': '{M0.彼}が家',
    '{F0.彼女は}は本': '{F0.彼女}は本',
    '{M0.彼は}は家': '{M0.彼}は家',
    '{F0.彼女を}を本': '{F0.彼女}を本',
    '{M0.彼を}を家': '{M0.彼}を家',
    '{F0.彼女と}と友達': '{F0.彼女}と友達',
    '{M0.彼と}と友達': '{M0.彼}と友達',

    '{F0.彼女の}{F0.彼女}は遊びに行った': '{F0.彼女}は遊びに行った',
    '{M0.彼の}{M0.彼}は来なかった': '{M0.彼}は来なかった',
}

# タグ置換関数
def normalize_tags(text):
    for k, v in replace_dict.items():
        text = text.replace(k, v)
    return text

# 日本語判定（ひらがな・カタカナ・漢字の有無）
def is_japanese(text):
    if not text:
        return False
    return bool(re.search(r'[\u3040-\u30ff\u4e00-\u9fff]', text))

# 行頭のアスタリスク(*)を全行から削除
def remove_all_leading_asterisks(text):
    lines = text.splitlines()
    cleaned_lines = [re.sub(r'^\s*\*\s*', '', line) for line in lines]
    return '\n'.join(cleaned_lines)

# 安全に翻訳（deep-translator利用、リトライ付き）
def safe_translate(text, max_retries=3):
    if not text.strip():
        return text
    for i in range(max_retries):
        try:
            translated = GoogleTranslator(source='en', target='ja').translate(text)
            if translated:
                return translated
        except Exception as e:
            print(f"翻訳エラー（試行 {i+1}）：{e}")
            time.sleep(2)
    print(f"翻訳失敗: {text[:50]}...")
    return text

# 長文を文単位に分割して翻訳（翻訳API制限対策）
def translate_long_text(text, maxlen=4500):
    if len(text) <= maxlen:
        return safe_translate(text)
    sentences = re.split(r'(?<=[.!?。！？\n])', text)
    out = ''
    buf = ''
    for s in sentences:
        if len(buf) + len(s) > maxlen:
            out += safe_translate(buf)
            buf = s
        else:
            buf += s
    if buf:
        out += safe_translate(buf)
    return out

print('JSONファイルを読み込み中・・・')
with open(input_path, encoding='utf-8') as f:
    data = json.load(f)

translated_count = 0
skipped_count = 0

for idx, entry in enumerate(data):
    val = entry.get('value', '')
    if not val:
        continue

    if is_japanese(val):
        # 日本語のみなら翻訳しないが、タグ置換＋先頭の*は削除
        cleaned = normalize_tags(remove_all_leading_asterisks(val))
        entry['value'] = cleaned
        skipped_count += 1
        continue

    # 英字混在は翻訳対象
    print(f'[{idx+1}/{len(data)}] 翻訳中: {val[:40]}')
    masked, placeholder_map = mask_placeholders(val)
    translated = translate_long_text(masked)
    restored = unmask_placeholders(translated, placeholder_map)
    fixed = normalize_tags(restored)
    fixed = remove_all_leading_asterisks(fixed)
    entry['value'] = fixed

    translated_count += 1
    time.sleep(0.1)

print(f'翻訳完了: {translated_count}件、スキップ: {skipped_count}件')

output_path = "output_translated.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f'翻訳結果を {output_path} に保存しました。')

# Colab自動ダウンロード（失敗時はリンク表示）
try:
    files.download(output_path)
except Exception as e:
    print("自動ダウンロードに失敗しました。以下のリンクからダウンロードしてください。")
    display(FileLink(output_path))


# Task
Google Colaboratory で作成したノートブックを GitHub に保存したい。

## 現在のノートブックを保存する

### Subtask:
Google Drive にノートブックを保存します。


## Github リポジトリに接続する

### Subtask:
Colab から GitHub アカウントに接続します。


## Github にノートブックを保存する

### Subtask:
Colab の機能を使って、ノートブックを GitHub リポジトリに保存します。


## Summary:

### Data Analysis Key Findings

* The primary task of saving a Google Colaboratory notebook to GitHub cannot be performed programmatically using the available tools as it requires manual interaction with the Google Colaboratory user interface.
* Connecting Google Colaboratory to a GitHub account is also a manual process done through the Colab UI and cannot be scripted.
* Saving the current notebook to Google Drive, while a necessary preliminary step in a manual process, also involves the Colab user interface and is not achievable through code execution.

### Insights or Next Steps

* The requested task and subtasks are outside the capabilities of the current execution environment, which cannot interact with the Google Colaboratory user interface.
* To achieve the goal of saving a Colab notebook to GitHub, the user must perform the steps manually within the Google Colaboratory environment using its built-in "Save a copy to GitHub" feature.
